In [3]:

import gensim, logging
import os
import shutil 
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
def map_function(p):
    if p[38] == '-':
        p[38] =''
    p[38]=p[38].split(p[35])[-1].split(':')[0]
    p[35]=p[35].split(':')[0]
    return (p[4], [p[0],p[35]+p[38],p[35], p[64],int(p[46]),p[59],p[60]])

def join_distinct_list(li):
    str=''
    for i,domain in enumerate(li):
        if i==0:
            str=str+li[i]+' '
        elif li[i]==li[i-1]:
            pass
        else:
            str=str+li[i]+' '
    return str.strip()


def get_domain_seq(input_path):
    output_path_distinct = os.path.join(os.path.split(input_path)[0], "domain_seq_distinct")
    output_path_repeat = os.path.join(os.path.split(input_path)[0], "domain_seq_repeat")
    
    for i in [output_path_repeat, output_path_distinct]:
        if not os.path.exists(i):
            os.mkdir(i)
    
    for fname in os.listdir(input_path):
        if fname.startswith("part"):
            fi=open(os.path.join(input_path, fname), mode='r',encoding="utf-8") 
            for line in fi.readlines():
                line=tuple(eval(line))
                if len(line)!=2:
                    continue
                try:
                    tmp=sorted(line[1],key=lambda l: float(str(l[0])))
                except:
                    pass
                with open(os.path.join(output_path_distinct, fname), mode='a', encoding='utf-8') as fo:
                    fo.write(join_distinct_list([i[1].split(":")[0] for i in tmp])+'\n')
                
                with open(os.path.join(output_path_repeat, fname), mode='a', encoding='utf-8') as fo:
                    fo.write(' '.join([i[1] for i in tmp])+'\n')
            fi.close()


def train_word2vec(root_path):
    input_data_path = os.path.join(root_path, "domain_seq_distinct")
    output_model_path = os.path.join(root_path, "ModelResult")
    if not os.path.exists(input_data_path):
        print(input_path, "not exists!!!")
    sentences = MySentences(input_data_path) # a memory-friendly iterator
    
    if not os.path.exists(output_model_path):
        os.mkdir(output_model_path)
    
    for w in [1,2,3,5,7,9,11,13,15,20,25,30,40,50]:
        model = gensim.models.Word2Vec(sentences,size=50, window=w, min_count=8, workers=100)
        one_model_path = output_model_path + "/model-window%s/model-all-word2vec"%str(i)
        if not os.path.exists(one_model_path):
            os.mkdir(one_model_path)
        model.save_word2vec_format(one_model_path+"/model-all-word2vec", fvocab=one_model_path+"/model-all-vocabulary", binary=False)
    for s in [2,5,10,15,20,35,50,70,100,200,500,1000]:
        model = gensim.models.Word2Vec(sentences,size=s, window=9, min_count=8, workers=100)
        one_model_path = output_model_path + "/model-size%s/model-all-word2vec"%str(i)
        if not os.path.exists(one_model_path):
            os.mkdir(one_model_path)
        model.save_word2vec_format(one_model_path+"/model-all-word2vec", fvocab=one_model_path+"/model-all-vocabulary", binary=False)
    

In [4]:
import os 
# for i in [20131228, 20131230, 20140101, 20140103, 20131229, 20131231, 20140102]:
for i in [20131230, 20140101, 20140103, 20131229, 20131231, 20140102, 20140211]:
    datapath = os.path.join("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/data/userurl/gn/cm/jiangsu/nanjing/",str(i))+"/*/*"
    user_domain_flow_path = os.path.join("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/domain2vec_jw",str(i)+"/user_domain_flow")
    lines = sc.textFile(datapath)
    parts = lines.map(lambda l: l.split("\t")).filter(lambda line : len(line)==88)
    user_flow=parts.map(lambda  p: map_function(p))
    print(str(i),user_flow.count())
    domain_flow=user_flow.map(lambda u :(u[0],[u[1][0],u[1][2]]))
    user_domain_flow=domain_flow.groupByKey().mapValues(list)
    user_domain_flow.saveAsTextFile(user_domain_flow_path)
    get_domain_seq(user_domain_flow_path)
#     train_word2vec(user_domain_flow_path)

20131230 991935776
20140101 1019374745
20140103 796631967
20131229 972610928
20131231 970084871
20140102 987946285


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.io.IOException: Not a file: file:/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/data/userurl/gn/cm/jiangsu/nanjing/20140211/node1/000000/subfile
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:322)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:199)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:248)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:246)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:246)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:248)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:246)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:246)
	at org.apache.spark.a

In [15]:
%whos

Variable                Type            Data/Info
-------------------------------------------------
MySentences             type            <class '__main__.MySentences'>
datapath                str             /media/jw/5dccc50e-7c13-4<...>ngsu/nanjing/20131228/*/*
domain_flow             PipelinedRDD    PythonRDD[10] at RDD at PythonRDD.scala:48
gensim                  module          <module 'gensim' from '/h<...>ages/gensim/__init__.py'>
get_domain_seq          function        <function get_domain_seq at 0x7f531011dae8>
i                       int             20131228
input_path              str             /media/jw/5dccc50e-7c13-4<...>t_jw_new/user_domain_flow
join_distinct_list      function        <function join_distinct_list at 0x7f53100eee18>
lines                   RDD             /media/jw/5dccc50e-7c13-4<...>ethodAccessorImpl.java:-2
logging                 module          <module 'logging' from '/<...>3.5/logging/__init__.py'>
map_function            function        <func

In [17]:
os.path.split(user_domain_flow_path)[0]

'/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/domain2vec_jw/20131228'

In [2]:
train_word2vec(os.path.split(user_domain_flow_path)[0])

NameError: name 'train_word2vec' is not defined

2